# Scoring benchmark


## Params

In [1]:
MUTATIONS = [0.8, 0.6, 0.4, 0.2, 0.1, 0.05, 0.01]
REMOVALS = [0.8, 0.6, 0.4 , 0.2, 0.1, 0.05, 0.01]
SEQUENCE_LENGTH = 1000

## Imports

In [ ]:
import sys
import os
import numpy as np

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from src.mutations import mutate_seq, random_base

In [3]:
!pip install numba

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Base sequences

In [ ]:
base_sequence = []
for j in range(SEQUENCE_LENGTH):
    base_sequence.append(random_base())
base_sequence = "".join(base_sequence)
# print(base_sequence, type(base_sequence))

mutated_sequences = []
for i in MUTATIONS:
    amount_mutations = int(SEQUENCE_LENGTH * i)
    new_seq, log = mutate_seq(base_sequence, amount_mutations)
    # print("amount_mutations",amount_mutations, "log", log,"new_seq type", type(new_seq))
    mutated_sequences.append(new_seq)

shortened_sequences = []
for j in REMOVALS:
    amount_removals = int(SEQUENCE_LENGTH * j)
    # We remove random positions from the base sequence
    shortened_seq = base_sequence
    for k in range(amount_removals):
        pos_to_remove = np.random.randint(0, len(shortened_seq))
        # Eliminar un carácter de un string usando slicing
        shortened_seq = shortened_seq[:pos_to_remove] + shortened_seq[pos_to_remove+1:]
    shortened_sequences.append(shortened_seq)



AAGGAGGAACAAGAGGGTGCGACATGAGTAACGAAAGACTCTTGAGGTCTTATCCGGGAGGGGATGTCTGGCCTCTATGGTCGACAAGGCTCTCGTCTCTGTGTTTCAGTCCGCGCTCGCTCACTCCCGCGGTAAAACGGTTTTTCAGTCGCCTACAAATCTATTGGAAGGAGCTCGGTGGAACTCAAAGCCCCCACGGATCCCTAGCCGGCAATATGGGTACTGCCTTTCGCGATCGGCACCATACTAGCAGTCACTTTTTGCATTAGGAGTCGTAAGCGTGGTGACGACAGTAGACCAGCGGAGACTGTATTCCCATCAGATGATTACGATGAGCCCATAGCACATTAGCTTGGAATATACCATGTGGACGCGAAAAAAGTCCGTCGGTAACATACATGCTAACCTCGGTACGACACAACCGAGCTTTTTATGCCAAGACGCCAGATATAGGGCCATAAGTGACCAACTGTCAAGCATCAGGGGGTAATTAGACCCTTCTAGCTCAGCGTATGTTCGGACAATTACGTGTCCCCTCTATCGTGGCAGCGCTGTAAAAGAGTCGGTTTCGGAGTTGCCAATCTCGTGGAGTCAGACGACTCCTTTTGGCGTGAAAAATAATTTGCGAGTAAGGGCTTTATGTCCTGCCCGTCTGAACTTAAATAATGGTAATGCGGAGTGCGGACATGTCGGTTACAACTACCCCTGAATCGTAATCAAGCGGGCGCTCTCAGCAAGGTAGCCCAACGAGCGGCCTATACGCACCACAGATTGTTCAATGAGATTGCGTGTGCCCAACTAAGAGGCAACGCAAGGCGAAGCCCCAGGAGATCACCCCGACGTGCCGGGACGGTGAAAATCGGTCGACTCGATGAACGTCATAGGACGCGAGGGCAAAGATCCTCAAAGCACAACCGCAATTCCTCCTCGGGGCTCCCTCGTATATTTGTGGCACTACCGCTCCCCGTTCTGCTCGGTCGACCCTAATTTAATATCGCTA

In [5]:
print(type(base_sequence))
print(type(mutated_sequences))
print(type(mutated_sequences[0]), f"<{mutated_sequences[0][0]}>")
print(type(shortened_sequences))

<class 'str'>
<class 'list'>
<class 'str'> <T>
<class 'list'>


## Import algorithims

In [6]:
from tests.algorithms_all import needleman_wunsch, needleman_wunsch_diag_numpy, needleman_wunsch_numba, needleman_wunsch_diagbounded_numpy

In [7]:
algorithims = [
    needleman_wunsch,
    needleman_wunsch_diag_numpy,
    needleman_wunsch_numba,
    needleman_wunsch_diagbounded_numpy
]

results = {}

for algorithm in algorithims:
    print(f"Benchmarking {algorithm.__name__} with mutations:")
    for seq in mutated_sequences:
        score = algorithm(base_sequence, seq)
        print(f"Score: {score[0]}",score[1], score[2])
        results.setdefault(algorithm.__name__, []).append(score[0])
    
    print(f"Benchmarking {algorithm.__name__} with removals:")
    for seq in shortened_sequences:
        score = algorithm(base_sequence, seq)
        print(f"Score: {score[0]}")
        results.setdefault(algorithm.__name__, []).append(score[0])

Benchmarking needleman_wunsch with mutations:
Score: 48 AAGGAGGAACAAGAG-GGTGCGACATGA-GTAACGAAAGACTCTTGAGGTCTTATCCGGGAGGGGATGTCTG-GCCTCTATGGTCGACAAGGCT-CTC-G-TCTC-TGTG-TTTCAGTCCGCGCTCGCT-CA-CTCC-C-G-CGGTAA-AACGGTTTTTCAGTCGCCTACAA-ATCT-ATTGGAAGGAGCTCG-GTGGAACT-CA--A-AGCCCCCAC-GGATCCCTAGCCGGCAATA-TGGGTACT-GCCTTTCGCGATC-GGCACCATACTAGCAGTCACTTTTTGCATTAGGAGTC-GTAAGCGTGGTGACGACAGTAGA-CCAGCGGAGACTGTA-TTCCCATCAGATGATTACGATGAGCCCATAGCACATTAGCTTGGAATATAC-CAT-G-TGGACGCGAAAAAAGTCCGT-CGGTA-ACATACAT-G-CTAACCTCGGTACGACACAACCGAGCTTTTTATGCC-AAGACGCCAGATATAGGGCCATA-AGTGA-CCAACTGTCAAGCA-TCAGGGGGTAATTA-GACCCTTCTAGCTCAGCGTATGTTCGGACAATTACGT-GTCCCCTCTATCGTGGCAGCGCTGTAAAAGAGTCGGTTTCG-GAGTTGCCAATCTCGT-GGAGTCAGACGACTCCTTTTGGC-GTGAAAAATAATTTGCG-AGTAAGGGC-T-TTAT-GTCCTG-CCCGTCTGAACTTAAATAATGGTAATGCGGAGTGCG-G-AC-ATGTCGGTTACAACTA-CCCCTGAATCGTAATCAAGCGGGCGCT-CTCAGCAAGGTAGCCCAACGAGCGGCCTATACGCACCACAGATTGTTC-A-ATGAGATTGCGTGTGCCCAACTAAGAGGCAACGCAAGGC-GAAGC-CCCA-GGAGATCAC-CCCGACGTG-CCG-GGACGGTGAAAATCG-GTCGACTCGAT-GA--ACG

## Compare results

In [8]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [9]:
import pandas as pd

# Recreate DataFrame from results (ensures consistent ordering)
df = pd.DataFrame.from_dict(results, orient='index')

# Build column names: first len(MUTATIONS) are mutation cases, next len(REMOVALS) are removal cases
mut_cols = [f"mut-{v}" for v in MUTATIONS]
rem_cols = [f"rem-{v}" for v in REMOVALS]
new_cols = mut_cols + rem_cols

# Handle potential length mismatch defensively
if len(new_cols) != df.shape[1]:
    if len(new_cols) > df.shape[1]:
        new_cols = new_cols[: df.shape[1]]
    else:
        extra = [f"case_{i+1}" for i in range(len(new_cols), df.shape[1])]
        new_cols = new_cols + extra

df.columns = new_cols

# Display the table
display(df.T)


,needleman_wunsch,needleman_wunsch_diag_numpy,needleman_wunsch_numba,needleman_wunsch_diagbounded_numpy
mut-0.8,48.0,48.0,48.0,48.0
mut-0.6,158.0,158.0,158.0,158.0
mut-0.4,338.0,338.0,338.0,338.0
mut-0.2,603.0,603.0,603.0,603.0
mut-0.1,791.0,791.0,791.0,791.0
mut-0.05,885.0,885.0,885.0,885.0
mut-0.01,978.0,978.0,978.0,978.0
rem-0.8,-1400.0,-1400.0,-1400.0,-1430.0
rem-0.6,-800.0,-800.0,-800.0,-808.0
rem-0.4,-200.0,-200.0,-200.0,-200.0
